<a href="https://colab.research.google.com/github/LordLean/Extracting-Green-Bonds-Use-of-Proceeds/blob/main/QA_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Retieval

## Answer Retriever


In [ ]:
!pip install rank-bm25

!pip install PyPDF2

!pip install tabula-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 208 kB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 5.2 MB/s 


In [ ]:
!wget https://www.amppartners.org/docs/default-source/investors/annual-reports/2020/2020_amp_sustainability_report.pdf

!wget https://www.icmagroup.org/Emails/icma-vcards/amp_combined%20hydro%20projects_External%20Review%20report.pdf

!wget https://www.globalworth.com/wp-content/uploads/2021/07/Globalworth-Green-Bond-Report-2020-20-July-2021.pdf

--2022-08-10 11:40:46--  https://www.amppartners.org/docs/default-source/investors/annual-reports/2020/2020_amp_sustainability_report.pdf
Resolving www.amppartners.org (www.amppartners.org)... 172.67.72.70, 104.26.14.54, 104.26.15.54, ...
Connecting to www.amppartners.org (www.amppartners.org)|172.67.72.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5034699 (4.8M) [application/pdf]
Saving to: ‘2020_amp_sustainability_report.pdf’

2020_amp_sustainabi 100%[===================>]   4.80M  1.37MB/s    in 3.4s    

2022-08-10 11:40:50 (1.43 MB/s) - ‘2020_amp_sustainability_report.pdf’ saved [5034699/5034699]

--2022-08-10 11:40:50--  https://www.icmagroup.org/Emails/icma-vcards/amp_combined%20hydro%20projects_External%20Review%20report.pdf
Resolving www.icmagroup.org (www.icmagroup.org)... 91.216.93.249
Connecting to www.icmagroup.org (www.icmagroup.org)|91.216.93.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 675346 (660K) [applic

In [ ]:
import numpy as np

import tabula
from rank_bm25 import BM25Okapi
from PyPDF2 import PdfReader

In [ ]:
class TableReader:

  def __init__(self, pdf):
    self.pdf = pdf
    self.dfs = None

  def read_pages(self, pages="all", multiple_tables=True, stream=True):
    '''
    Return tables discovered within pdf.
    '''
    self.dfs = tabula.read_pdf(self.pdf, pages=pages, multiple_tables=multiple_tables, stream=stream)
    self.__clean_dfs()
    return self.dfs

  def __clean_dfs(self, thresh=2):
    self.dfs = [df.dropna(thresh=thresh) for df in self.dfs]


class Reader:

  def __init__(self, filename):
    self.reader = PdfReader(filename)
    self.tb = TableReader(filename)
    self.page_viewer = {page_num : {} for page_num in range(self.reader.numPages)}
    self.idx2page_item = []
  
  def __extract_text(self,):
    '''
    Page-wise text extraction and tokenize for BM25.
    '''
    text_index_mem = 0
    # List to store each tokenized corpus
    tokenized_corpus_list = []
    for i in range(self.reader.numPages):
      raw_text = self.reader.getPage(i).extractText()
      self.page_viewer[i]["raw_text"] = raw_text
      # Split text
      corpus = raw_text.split("\n \n")
      # Store results.
      self.page_viewer[i]["corpus"] = corpus
      for item in corpus:
        self.idx2page_item.append((i, item)) # page,textItem
      # Tokenize
      tokenized_corpus = [doc.split(" ") for doc in corpus]
      tokenized_corpus_list.append(tokenized_corpus)
    # BM25 computations only after the complete tokenized corpus is collated. 
    # Merge tokenized corpus'.
    tokenized_corpus_complete = [item for sublist in tokenized_corpus_list for item in sublist]
    # BM25
    self.bm25 = BM25Okapi(tokenized_corpus_complete)

  def __extract_tables(self):
    '''
    Page-wise table extractor.
    '''
    for i in range(self.reader.numPages):
      # page=0 will throw error using tabula.
      page = str(i+1)
      self.page_viewer[i]["tables"] = self.tb.read_pages(pages=page)

  def extract_pdf(self):
    # Extract data
    self.__extract_text()
    self.__extract_tables()

  def print_page(self, page_num):
    '''
    Print separated sections of text given a page.
    '''
    corpus = self.page_viewer[page_num]["corpus"]
    for item in (corpus):
      print("\n{}\n".format("-"*60))
      print(item)
    print("\n{}\n".format("-"*60))
    for df in self.page_viewer[page_num]["tables"]:
      print(df.style)
      display(df)

  def __score(self, queries, weights):
    '''
    Compute the average BM25 score of each given query on each page of text.
    '''
    self.ranked_scores = []
    for query in queries:
      # tokenize query by whitespace.
      tokenized_query = query.split()
      # Compute score.
      doc_scores = self.bm25.get_scores(tokenized_query)
      self.ranked_scores.append(doc_scores)
    # Compute average (weighted) score against all queries.
    if not len(weights):
      # Equal weighting.
      self.average_score = np.average(self.ranked_scores, axis=0)
    elif len(queries) != len(weights):
        # Unequal number of elements.
        raise ValueError("Number of query and weight elements passed must be equal.")
    else:
      # Weighted average.
      self.average_score = np.average(self.ranked_scores, weights=weights, axis=0)
 
  def get_ranked_texts(self, queries, weights=[], n=5):
    '''
    Return n pages which scored highest using BM25.
    '''
    # Run score method to calculate BM25.
    self.__score(queries, weights)
    idx = sorted(range(len(self.average_score)), key=lambda i: self.average_score[i], reverse=True)[:n]

    final_results = []
    for i in range(n):
      page_num, text = self.idx2page_item[idx[i]]
      tables = self.page_viewer[page_num]["tables"]
      final_results.append({"page_num":page_num, "text":text, "tables":tables})

    return final_results
    

In [ ]:
filename = "Globalworth-Green-Bond-Report-2020-20-July-2021.pdf"
reader = Reader(filename)

reader.extract_pdf()
queries = [
    "use of proceeds",
    "allocation of proceeds",
    # "projects financed"
    ]

top_n = 5
top_items = reader.get_ranked_texts(queries, n=top_n)

2022-08-10 11:41:17 [WARNING] io: Got stderr: Aug 10, 2022 11:41:14 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
Aug 10, 2022 11:41:14 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Aug 10, 2022 11:41:14 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>

2022-08-10 11:41:44 [WARNING] io: Got stderr: Aug 10, 2022 11:41:41 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 10, 2022 11:41:44 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

2022-08-10 11:41:54 [WARNING] io: Got stderr: Aug 10, 2022 11:41:50 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 10, 2022 11:41:53 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

2022-08-10 11:42:22 [WARNING] io: Got stderr: Aug 10, 2022 11:42:21 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 10, 2022 11:42:22 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [ ]:
for item in top_items:
  page_num = item["page_num"]
  text = item["text"]
  tables = item["tables"]
  print("Page: {}\n\n{}".format(page_num,text))
  for table in tables:
    display(table.style)
  print("-"*60)
  print("\n\n")

Page: 4

Globalworth in line with its commitment under it Green Bond Framework, to enable investors to 
follow its Green Bond progress, and to provide insight to prioritised areas, is providing this Green Bond 
update consisting of an Allocation Report and an Impact Report (where feasible) . 
The allocation of the green bond proceeds and compliance with the financ ing  or refinanc ing of  Eligible 
Green Projects  is subject to an annual external assurance by an independent third party, ERNST & 
YOUNG (HELLAS) Certified Auditors - Accountants S.A. (EY ), (as appended hereto).  

------------------------------------------------------------



Page: 4

Sustainal ytics Highlights on Globalworths’ Green Bond Framework (28 May 2020)  
Use of Proceeds:  • The eligible categories for the use of proceeds, Green Buildings and 
Energy Efficiency, are aligned with those recognized by the Green Bond 
Principles 2018.  
• Sustainalytics considers that the eligible categories wi ll lead to positive 

,Pre-screening based,Unnamed: 0,Unnamed: 1,Unnamed: 2,External review of,Unnamed: 3
0,nan,nan,Final validation by the,nan,Eligible Use of,nan
2,criteria by investment,nan,Green Bond,nan,Proceeds with the,nan
3,nan,nan,Committee (annually,nan,criteria displayed in,nan
5,nan,nan,or earlier if necessary),nan,the Green Bond,nan
9,nan,Eligible,nan,Validated,nan,Verified
10,STEP 1,assets,STEP 2,nan,STEP 3,assets


------------------------------------------------------------



Page: 9

6. Allocation of Proceeds  
Globalworth’s net proceeds from its inaugural Green Bond issue in July 2020  were €386.5 million, of 
which €376.9 million or c. 97. 5% have been allocated in standing and under 
refurbishment/ construction  properties. The remainder unallocated proceeds of € 9.6  million, will be 
dedicated to financing an office  project  currently under construction  which is expected to be delivered  
in 2021.  
The Green Bond Committee decided to allocate the proceeds as follows:  
Allocations:        
 Country  No of 
Buildings  Status  Certification Level  Allocated  
Amounts 
(€m)  Unallocated  
Amounts  
(€m)  
Globalworth Campus  Romania  3 Standing  BREEAM Excellent  198.4  - 
Globalworth Square  Romania  1 Under  
Con struc tion   [BREEAM Outstanding]  40.0  9.6  
Podium Park  A Poland  1 Standing  BREEAM Outstanding  40.3  - 
Renoma  Poland  1 Under 
Refurbishment  BREEAM Excellent  98.3  -

,Allocations:,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,nan,Country,No of,Status,Certification Level,Allocated,Unallocated
1,nan,nan,Buildings,nan,nan,Amounts,Amounts
2,nan,nan,nan,nan,nan,(€m),(€m)
3,Globalworth Campus,Romania,3,Standing,BREEAM Excellent,198.4,-
4,Globalworth Square,Romania,1,Under,[BREEAM Outstanding],40.0,9.6
6,Podium Park A,Poland,1,Standing,BREEAM Outstanding,40.3,-
7,Renoma,Poland,1,Under,BREEAM Excellent,98.3,-
9,TOTAL:,nan,nan,nan,nan,376.9,9.6


------------------------------------------------------------



Page: 6

• Individual measures : Individual measures reducing energy use and/or 
carbon emissions for the operational phase of the building. A list of 
eligible indivi dual measures can be found under Appendix 1 of the Green 
Bond Framework.  


,Pre-screening based,Unnamed: 0,Unnamed: 1,Unnamed: 2,External review of,Unnamed: 3
0,nan,nan,Final validation by the,nan,Eligible Use of,nan
2,criteria by investment,nan,Green Bond,nan,Proceeds with the,nan
3,nan,nan,Committee (annually,nan,criteria displayed in,nan
5,nan,nan,or earlier if necessary),nan,the Green Bond,nan
9,nan,Eligible,nan,Validated,nan,Verified
10,STEP 1,assets,STEP 2,nan,STEP 3,assets


------------------------------------------------------------





## Answer Re-ranker (Neural: BERT / T5)

In [ ]:
!pip install pygaggle

!pip install transformers==4.6.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 914 kB/s 
     |████████████████████████████████| 7.3 MB 10.2 MB/s 
     |████████████████████████████████| 63.3 MB 30 kB/s 
     |████████████████████████████████| 10.6 MB 38.2 MB/s 
     |████████████████████████████████| 2.9 MB 35.5 MB/s 
     |████████████████████████████████| 60 kB 4.3 MB/s 
     |████████████████████████████████| 1.4 MB 34.0 MB/s 
     |████████████████████████████████| 1.1 MB 26.2 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 8.6 MB 17.6 MB/s 
     |████████████████████████████████| 1.3 MB 36.4 MB/s 
     |████████████████████████████████| 184 kB 54.5 MB/s 
     |████████████████████████████████| 2.2 MB 57.2 MB/s 
     |████████████████████████████████| 3.7 MB 40.8 MB/s 
     |████████████████████████████████| 880 kB 57.1 MB/s 
  Created wheel for pygaggle: filenam

In [ ]:
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5, MonoBERT

class Reranker:

  def __init__(self):
    self.mono5t = MonoT5()
    self.monobert = MonoBERT()

  def rerank(self, query, texts, method="T5"):
    query = Query(query)
    texts = [Text(text, {"docid" : i}, 0) for i, text in enumerate(texts)]

    if method == "T5":
      reranker = self.mono5t
    if method == "BERT":
      reranker = self.monobert

    reranked = reranker.rerank(query, texts)
    reranked.sort(key=lambda x: x.score, reverse=True)

    return reranked

In [ ]:
reranker = Reranker()

query = "what did the use of proceeds finance"

texts = [item["text"] for item in top_items]

reranked = reranker.rerank(query, texts, method="T5")

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


In [ ]:
# Print out reranked results:
for i in range(len(reranked)):
  print(i)
  print("Score: {}".format(reranked[i].score))
  print("Text:\n{}".format(reranked[i].text))
  print("\n\n{}".format("-"*60))

0
Score: -0.011511215940117836
Text:
Sustainal ytics Highlights on Globalworths’ Green Bond Framework (28 May 2020)  
Use of Proceeds:  • The eligible categories for the use of proceeds, Green Buildings and 
Energy Efficiency, are aligned with those recognized by the Green Bond 
Principles 2018.  
• Sustainalytics considers that the eligible categories wi ll lead to positive 
environmental impacts and advance the UN Sustainable Development 
Goals, specifically SDGs 7 & 11  
Project Evaluation / 
Selection:  • Globalworth’s  internal process of evaluating and selecting projects is 
carried out by the Green Bond Committee. The Committee is responsible 
for screening projects against the eligibility criteria and recommending 
eligible projects for inclusion in the Eligible Green P roject Portfolio. The 
Portfolio will be reviewed annually to ensure projects’ eligibility and, if 
no longer eligible, projects will be removed and replaced as soon as 
practically feasible.  
• Sustainalytics 

# QA Model

In [3]:
# Download zipped model
!gdown 1NBc9MfT4FuchadevFJvcfyBfpIPMILb0

# Unzip
!unzip finbert-pretrain-finetuned-squad.zip

Downloading...
From: https://drive.google.com/uc?id=1NBc9MfT4FuchadevFJvcfyBfpIPMILb0
To: /content/finbert-pretrain-finetuned-squad.zip
100% 406M/406M [00:06<00:00, 61.1MB/s]


In [8]:
!pip install transformers 

from transformers import pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.0 MB/s 
     |████████████████████████████████| 101 kB 10.0 MB/s 
     |████████████████████████████████| 596 kB 39.1 MB/s 
     |████████████████████████████████| 6.6 MB 35.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [9]:
model_dir = "finbert-pretrain-finetuned-squad"

question_answering = pipeline("question-answering", model=model_dir, tokenizer=model_dir)

In [14]:
context = """
Machine learning (ML) is the study of computer algorithms that improve automatically through experience. It is seen as a part of artificial intelligence. Machine learning algorithms build a model based on sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to do so. Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or unfeasible to develop conventional algorithms to perform the needed tasks.
"""
question = "what are the machine learning models based on?"

question, context = "Who was Jim Henson?", "Jim Henson was a nice puppet"

In [15]:
result = question_answering(question=question, context=context, device=0)
print("Answer:", result['answer'])
print("Score:", result['score'])

Answer: a nice puppet
Score: 0.49829721450805664
